# Part Three - Data Munging (cleaning, joining, manipulating)

"Data munging" refers to a whole host of techniques and processes around loading, joining, and cleaning data. In this tutorial we'll cover a few of those techniques, including some more detail on reading csvs, and a couple of ways of joining DataFrames together. Some of the techniques in this notebook use a common Python expression, the `for` loop. You might want to read up on that to get some familiarity with it, if you've not encountered it before.


In [1]:
import pandas as pd

## Reading CSVs

We've already looked at the basics of reading a csv, but there are a few other things that are super useful when you're reading messy data.

Let's look at a new csv.

In [2]:
# Data is from: https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi

url = 'https://raw.githubusercontent.com/SimonCarryer/pandas_tutorial/master/data/pa_2007_irs.csv'
income = pd.read_csv(url) 
income.head()

,Zip Code,Number of Returns,Adjusted gross income (AGI)
0,TOTAL,"6,601,648","343,163,849"
1,15001,"18,306","732,875"
2,15003,"6,639","205,884"
3,15004,196,"6,735"
4,15005,"5,546","299,894"


Doesn't look too bad! But there's some hidden problems that might come back to bite us:
* The column names are very long and have inconsistent capitalisation. That's going to be annoying to work with.
* The values have commas separating the thousands, which means Pandas will treat them as strings.
* The Zip Code column has a "TOTAL" value in it, but also numeric values, which means that the whole column has mixed data types - this confuses Pandas.
* The "TOTAL" row is going to be annoying for other reasons - it'll skew any sums or averages we get from the DataFrame.

Let's see what we can do about those problems.

In [3]:
income = pd.read_csv(url, 
                 thousands=',',                 # The 'thousands' argument tells Pandas to look for commas and remove them
                 index_col=0,                   # Since there's only one row for each zip, let's use that column as the index
                 names=['n_returns', 'income'], # We'll supply our own column names for our DataFrame
                 skiprows=2             # Since we're supplying our own headers, and we don't want the TOTAL row, let's skip 'em
                )
income.head()

,n_returns,income
15001,18306,732875
15003,6639,205884
15004,196,6735
15005,5546,299894
15006,227,8072


Much nicer. The `read_csv` function has a lot of different knobs and dials you can fiddle with to clean your data as you're reading it. I suggest reading [the documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) any time you're dealing with a nasty csv - you might find just what you need.

Oh! It's also worth noting that the DataFrame we have now is a bit different to the ones we've seen before. Its index isn't just the row number, it actually holds some information about the row. We can still access rows just like we did before, like this,

In [4]:
income.iloc[:5]

,n_returns,income
15001,18306,732875
15003,6639,205884
15004,196,6735
15005,5546,299894
15006,227,8072


But if we have a particular zip code we're interested in, we can access it using the `loc` function.

In [5]:
income.loc[15004]

n_returns     196
income       6735
Name: 15004, dtype: int64

That's going to come in handy later.

### Problems

* Load the `dog_registrations.csv` file (from 'https://raw.githubusercontent.com/SimonCarryer/pandas_tutorial/master/data/dog_registrations.csv' or your local file system), but use the 'Breed' column as the index
* Load the `dog_registrations.csv` file, but supply your own column names
* EXTRA FOR EXPERTS: Load the `dog_registrations.csv` file, but parse the 'ValidDate' column as a date during the load

## Concatenating Data

Concatenating is for when you have two dataframes that share a dimension - the same index, or the same columns - and you want to stick 'em together along that shared dimension. The most common case, in my experience, is when you have several "pages" of the same table - the same columns but different rows, and you want to put them all into one `DataFrame`. It's effectively the same as a SQL `UNION ALL` statement.

Here's an example where we've got several csvs, and we want to patch them together into a single `DataFrame`. First, we'll load each of three DataFrames.

In [6]:
url2 = 'https://raw.githubusercontent.com/SimonCarryer/pandas_tutorial/master/data/pa_2008_irs.csv'
url3 = 'https://raw.githubusercontent.com/SimonCarryer/pandas_tutorial/master/data/pa_2009_irs.csv'

income_2007 = pd.read_csv(url, thousands=',', index_col=0, names=['n_returns', 'income'], skiprows=2)
income_2008 = pd.read_csv(url2, thousands=',', index_col=0, names=['n_returns', 'income'], skiprows=2)
income_2009 = pd.read_csv(url3, thousands=',', index_col=0, names=['n_returns', 'income'], skiprows=2)

We want to preserve the year of each DataFrame, so let's add that data:

In [7]:
income_2007['year'] = 2007
income_2008['year'] = 2008
income_2009['year'] = 2009

In [8]:
len(income_2009)

1370

We can glue all three of these together by putting them in a list, and passing that list to Pandas' `concat` function, like this.

In [9]:
list_of_dataframes = [income_2007, income_2008, income_2009]

income = pd.concat(list_of_dataframes)

In [10]:
income['year'].value_counts()

2007    2010
2008    1399
2009    1370
Name: year, dtype: int64

But there's a heap of repeated lines of code there, where we read each csv into a DataFrame. Also, we're storing each csv in a seperate DataFrame, which will stay in memory. That's fine for these little files, but if the files were really big, or if there were heaps of them, this would become a problem.

Let's see if we can do that a little more concisely, by looping through a list of years.

In [11]:
list_of_dataframes = []

for year in [2007, 2008, 2009]:
    url = f'https://raw.githubusercontent.com/SimonCarryer/pandas_tutorial/master/data/pa_{year}_irs.csv'
    temp = pd.read_csv(url, thousands=',', index_col=0, names=['n_returns', 'income'], skiprows=2)
    temp['year'] = year
    list_of_dataframes.append(temp)
income = pd.concat(list_of_dataframes)

That's a wee bit obscure. We've looped through each year in a list of three years, and for each one, gone and loaded that csv, added the year column, and put the dataframe into a list. Finally we pass all three DataFrames to the `concat` function, which gives us a single `DataFrame`.

In [12]:
income.head()

,n_returns,income,year
15001,18306,732875,2007
15003,6639,205884,2007
15004,196,6735,2007
15005,5546,299894,2007
15006,227,8072,2007


In [13]:
income['year'].value_counts()

2007    2010
2008    1399
2009    1370
Name: year, dtype: int64

I wonder why there are more zip codes in 2007 compared to the subsequent two years?

#### A NOTE ABOUT `for` LOOPS

`for` loops are great! They let you iterate through a list and do something with each element. Experienced programmers are used to thinking in terms of loops. But! Loops can get you into trouble in Pandas. For example, you might be tempted to do something like this.

In [14]:
for year in [2007, 2008, 2009]:
    this_years_income = income.loc[income['year'] == year]
    print(year, this_years_income['income'].mean())

2007 170728.2895522388
2008 235782.92351679772
2009 232769.29416058393


And for a small DataFrame, that's fine! But if you make a habit of this, sooner or later you're going to hit a big DataFrame, where looping in this way causes serious performance issues. It's always better, if you can, to find a way to use the native functions that Pandas provides. These are optimised to work over large datasets and will make your code run much, much faster.

In [15]:
# Do this instead

income.groupby('year')['income'].mean()

year
2007    170728.289552
2008    235782.923517
2009    232769.294161
Name: income, dtype: float64

### Concatenating Row-wise

Above, we concatenated columns together, like a SQL `UNION ALL`. But we can aslo use `concat` to stick rows together, more like a SQL `OUTER JOIN`. Like many things in Pandas, this relies on having a `DataFrame` or `Series` with aligned indexes.

Let's try calculating the average income for each zip code, and making a `DataFrame` with one column for each year. First, let's get the average income for one year.

In [16]:
average_income_2007 = income_2007['income'] / income_2007['n_returns']
average_income_2007.name = 2007

In [17]:
average_income_2007.head()

15001    40.034688
15003    31.011297
15004    34.362245
15005    54.073927
15006    35.559471
Name: 2007, dtype: float64

Note how the index for the `Series` comes from the index of the `DataFrame` we've derived it from.

Now let's get another year's data.

In [18]:
average_income_2008 = income_2008['income'] / income_2008['n_returns']
average_income_2008.name = 2008

In [19]:
average_income_2008.head()

15001    44.862339
15003    35.516510
15005    57.256456
15009    56.838348
15010    45.513009
Name: 2008, dtype: float64

It looks like the indexes align, but on closer investigation, you can see that 2008 is missing some zip codes that are included in 2007. That's going to have an interesting effect when we concatenate the data.

In [20]:
average_income_by_year = pd.concat([average_income_2007, average_income_2008], axis=1) # Note the 'axis=1' which makes this work.

In [21]:
average_income_by_year.head(10)

,2007,2008
15001,40.034688,44.862339
15003,31.011297,35.516510
15004,34.362245,NaN
15005,54.073927,57.256456
15006,35.559471,NaN
15007,44.422414,NaN
15009,50.277188,56.838348
15010,40.697321,45.513009
15012,43.066313,49.067837
15014,32.240997,35.211387


The `Series` objects have been joined along their indexes. Where a zip code is missing in one of the Series, the DataFrame is populated with a `NaN` value, like a SQL `NULL`.

Like before, there's a more concise way of achieving this result.

In [22]:
list_of_series = []

for year in [2007, 2008, 2009]:
    url = f'https://raw.githubusercontent.com/SimonCarryer/pandas_tutorial/master/data/pa_{year}_irs.csv'
    temp = pd.read_csv(url, thousands=',', index_col=0, names=['n_returns', 'income'], skiprows=2)
    average_income = temp['income'] / temp['n_returns']
    average_income.name = year
    list_of_series.append(average_income)
average_income_by_year = pd.concat(list_of_series, axis=1)

In [23]:
average_income_by_year.head(10)

,2007,2008,2009
15001,40.034688,44.862339,NaN
15003,31.011297,35.516510,35.732487
15004,34.362245,NaN,NaN
15005,54.073927,57.256456,57.972025
15006,35.559471,NaN,NaN
15007,44.422414,NaN,51.538095
15009,50.277188,56.838348,55.226807
15010,40.697321,45.513009,44.916470
15012,43.066313,49.067837,49.452695
15014,32.240997,35.211387,33.777569


### Problems


* Make a `DataFrame` that has a row for each year, and a column for the count, max, min, and average income for each year
* HARD MODE: There's another file, `pa_2010_irs_raw.csv`, but I haven't tidied it up as much. Can you fix it up and append it to the other information?


## Joining across DataFrames

The `concat` function is great when you have nicely aligned indexes or columns, but gets confusing when you are joining on columns, not indexes, when you have multiple join criteria or there are complex join conditions. In those cases, there are a couple of other approaches which are better for these cases.

Let's get our old dog registrations dataset to experiment with.

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/SimonCarryer/pandas_tutorial/master/data/dog_registrations.csv')

df['ValidDate'] = pd.to_datetime(df['ValidDate'])

df.head()

,LicenseType,Breed,Color,DogName,OwnerZip,ExpYear,ValidDate
0,Dog Individual Female,AM PIT BULL TERRIER,SPOTTED,BUTTER,15001,2007,2007-05-01 15:15:00
1,Dog Individual Female,AM PIT BULL TERRIER,BROWN,SABLE,15001,2007,2007-05-01 15:15:00
2,Dog Individual Neutered Male,MIXED,.,YIP,15001,2007,2007-04-11 15:14:00
3,Dog Individual Male,DOBERMAN PINSCHER,RED,SABER,15003,2007,2007-04-05 15:00:00
4,Dog Individual Spayed Female,MIXED,BLACK,DAISY,15003,2007,2007-05-25 12:15:00


One of the easiest ways to join data across DataFrames is to use the `Series` function `map`. This function takes a `Series`, and for each item, applies a lookup fucntion to find the appropriate value for that row. The most common case is to map a series to a dictionary of key: value pairs. A neat trick about the `Series` object is that it acts kind of like a dictionary, with the index as the keys.

Let's find the average income for each year, using the `groupby` function we learned earlier.

In [25]:
grouped = income.groupby('year').sum()  # since we want to sum both columns in the DataFrame, we don't need to specify a column
yearly_income = grouped['income']/grouped['n_returns']
yearly_income

year
2007    51.981545
2008    55.087950
2009    54.685337
dtype: float64

Now we can use the `map` function to join that information into our dog registrations dataset.

In [26]:
df['Income for Year'] = df['ExpYear'].map(yearly_income)

In [27]:
df.iloc[[1, 40000, 80000]]

,LicenseType,Breed,Color,DogName,OwnerZip,ExpYear,ValidDate,Income for Year
1,Dog Individual Female,AM PIT BULL TERRIER,BROWN,SABLE,15001,2007,2007-05-01 15:15:00,51.981545
40000,Dog Individual Male,GOLDEN RETRIEVER,GOLD,RUSTY,15104,2008,2008-03-31 13:31:00,55.087950
80000,Dog Individual Spayed Female,AM PIT BULL TERRIER,WHITE/BROWN,JAZMIN,15132,2009,2009-02-19 10:20:00,54.685337


That's cool, but what we really want to know is the income for that zip code, for that year. To do that we'll need to join on multiple conditions - the right zip code, and the right year. How do we do that?

The best way, in my opinion, is to use Pandas' `merge` function. You describe the list of columns from each `DataFrame` that will be used to join the two sets of data. This uses some SQL-like concepts, where one table is the "left", and the other is the "right". Like some of the other functions we've seen, it accepts either the name of a column in the DataFrame, or else any other `Series` that shares the same index.

Here's how that looks, joining our dog registrations to the concatenated income DataFrame from earlier.

In [28]:
pd.merge(df,                              # This is the 'left' DataFrame
         income,                          # This is the 'right' DataFrame
         left_on=['OwnerZip', 'ExpYear'], # This is the list of columns we're joining on from 'df'
         right_on=[income.index, 'year']  # Note that we're joining on the index of 'income', as well as the 'year' column
        )

,LicenseType,Breed,Color,DogName,OwnerZip,ExpYear,ValidDate,Income for Year,n_returns,income,year
0,Dog Individual Female,AM PIT BULL TERRIER,SPOTTED,BUTTER,15001,2007,2007-05-01 15:15:00,51.981545,18306,732875,2007
1,Dog Individual Female,AM PIT BULL TERRIER,BROWN,SABLE,15001,2007,2007-05-01 15:15:00,51.981545,18306,732875,2007
2,Dog Individual Neutered Male,MIXED,.,YIP,15001,2007,2007-04-11 15:14:00,51.981545,18306,732875,2007
3,Dog Individual Male,DOBERMAN PINSCHER,RED,SABER,15003,2007,2007-04-05 15:00:00,51.981545,6639,205884,2007
4,Dog Individual Spayed Female,MIXED,BLACK,DAISY,15003,2007,2007-05-25 12:15:00,51.981545,6639,205884,2007
5,Dog Individual Neutered Male,MIXED,SPOTTED,SCOOTER,15003,2007,2007-06-19 12:13:00,51.981545,6639,205884,2007
6,Dog Individual Spayed Female,RAT TERRIER,MULTI,TINKY,15003,2007,2007-07-13 13:35:00,51.981545,6639,205884,2007
7,Dog Individual Female,GER SHEPHERD,BLACK/BROWN,AMICA,15003,2007,2007-02-27 11:52:00,51.981545,6639,205884,2007
8,Dog Senior Citizen or Disability Spayed Female,POMERANIAN,TAN,TAFFY,15003,2007,2007-03-12 15:57:00,51.981545,6639,205884,2007
9,Dog Individual Spayed Female,BEAGLE,SPOTTED,BELLE,15003,2007,2007-01-26 09:24:00,51.981545,6639,205884,2007


### Problems

* In the `income` DataFrame, make another column that shows, for each zip code, the average income across all three years
* Add a new column to the dog registrations DataFrame, which is the average income (income over n_returns) for that zip and year